# Generate SMA using daily data service - Python

### Overview
Moving averages smooth the price movements by filtering out the "noise" caused by short-term price fluctuations.They also form the building blocks for a wide range of technical indicators.

Computing moving averages over a defined number of time periods is useful to:
* identify the trend direction
* determine support and resistance levels

This sample demonstrates how to request and plot **on-demand** *simple moving average - SMA* from a `daily data service`.

### Inputs/outputs
Simple moving averages' requests require instrument's identifier, date time intervals and number of periods as per inputs.

Basically, SMA is formed by computing the average price of a security over a specific number of points. As its name implies, simple moving average is an average that moves. Old data is dropped and new data becomes available, causing the average to move along the time.

N-points SMA data point is computed by adding up the last N period's trade prices and then dividing by N as follows:

$$ SMA = \frac{\Sigma Close_n}{N} $$
Where:
* *$Close_n$* is the close price of the *period n*
* *N* is the count of time periods

This sample shows how to plot a simple graph for basis technical analysis using an open source library. 

### Services used
This sample uses *gRPC requests* in order to retrieve daily data from the hosted service. The queried endpoint in this script are:
* *DailySmaService*: to directly retrieve daily SMA data from the server

### Modules required
1. Systemathics:
    * *systemathics.apis.services.daily_analytics.v1*
    * *systemathics.apis.type.shared.v1*
    * *google.type*
2. Open source:
    * *googleapis-common-protos*
    * *protobuf*
    * *grpcio*
    * *pandas*
    * *plotly* and *matplotlib* as per display package

***

# Run daily SMA sample

### Step 1: Install packages and import them

In [ ]:
pip install googleapis-common-protos protobuf grpcio pandas plotly matplotlib systemathics.apis

In [ ]:
import os
import grpc
import pandas as pd
from datetime import datetime
import google.type.date_pb2 as date
import systemathics.apis.type.shared.v1.identifier_pb2 as identifier
import systemathics.apis.type.shared.v1.constraints_pb2 as constraints
import systemathics.apis.type.shared.v1.date_interval_pb2 as dateinterval
import systemathics.apis.services.daily_analytics.v1.daily_sma_pb2 as daily_sma
import systemathics.apis.services.daily_analytics.v1.daily_sma_pb2_grpc as daily_sma_service
import systemathics.apis.helpers.token_helpers as token_helpers
import systemathics.apis.helpers.channel_helpers as channel_helpers

### Step 2: Retrieve authentication token
The following code snippet sends authentication request and print token to console output in order to process the upcomming *gRPC queries*.

In [ ]:
token = token_helpers.get_token()
display(token)

### Step 3: Retrieve data
To request *daily sma* service, we need to specify:
* Instrument identifier
* Time period selection: select start and end dates
* The SMA parameters

#### 3.1 Request Parameters

In [ ]:
# set instrument identifier: exchange + ticker
ticker = 'AAPL'
exchange = 'XNGS'

#### 3.2 SMA parameters

In [ ]:
# set SMAs lengths
long_sma = 200;
short_sma = 50;

#### 3.3 Time period selection

In [ ]:
# set data interval (we are using Google date time format) 
date_interval = dateinterval.DateInterval(
    start_date = date.Date(year = 2019, month = 1, day = 1),
    end_date = date.Date(year = 2020, month = 12, day = 31)
)

# generate constraints based on the previous date selection
# time constraints are not neeeded for the dividends
constraints = constraints.Constraints(
    date_intervals = [date_interval]
)

#### 3.4 EMAs request creation
The following code snippet creates *gRPC client*, process daily SMA requests and returns the reply.

In [ ]:
# generate daily long sma request
long_sma_request = daily_sma.DailySmaRequest(
    identifier = identifier.Identifier(exchange = exchange, ticker = ticker), 
    length = long_sma,
    constraints = constraints
)

# generate daily short sma request
short_sma_request = daily_sma.DailySmaRequest(
    identifier = identifier.Identifier(exchange = exchange, ticker = ticker),
    length = short_sma,
    constraints = constraints
)

In [ ]:
try:
    # open a gRPC channel
    with channel_helpers.get_grpc_channel() as channel:  
        
        # instantiate the daily sma service
        service = daily_sma_service.DailySmaServiceStub(channel)
        
        # Process the daily long SMA request
        long_sma_response = service.DailySma(
            request = long_sma_request, 
            metadata = [('authorization', token)]
        )
        
        # process the daily short SMA request
        short_sma_response = service.DailySma(
            request = short_sma_request, 
            metadata = [('authorization', token)]
        )
except grpc.RpcError as e:
    display(e.code().name)
    display(e.details())

The following code snippet displays total data points count:

In [ ]:
print("Total data points retrieved for long SMA: ", len(long_sma_response.data))
print("Total data points retrieved for short SMA: ", len(short_sma_response.data))

### Step 4: Visualize data

#### 4.1 Retrieve SMAs data
In the following code snippets, the daily SMA reply is reprocessed in a dataframe in order to visualize the results:

In [ ]:
# create sma values list, set nan null values to avoid zero-values plot
length = len(long_sma_response.data)
shorts, longs= [None]*length, [None]*length

for i in range(length):
    # get non null sma values
    long = long_sma_response.data[i].average.value
    short = short_sma_response.data[i].average.value
    
    # store non null sma values 
    if  long != 0:
        longs[i] = long
    if short != 0:
        shorts[i] = short

In [ ]:
# retrieve dates and daily prices
dates = [datetime(l.date.year, l.date.month, l.date.day) for l in long_sma_response.data]
prices = [l.value for l in long_sma_response.data]

In [ ]:
# create a pandas dataframe with dates, prices and smas
d = {'Date': dates, 'Short sma': shorts, 'Long sma': longs, 'Price': prices}
df = pd.DataFrame(data=d)
df

#### 4.2 Plot SMAs and daily prices with matplotlib

In [ ]:
import matplotlib.pyplot as plt

# create figure and axis objects with subplots()
fig,ax = plt.subplots(1,1,figsize=(25,10))
ax.plot('Date', 'Long sma', data=df, marker='', color='orange', linewidth=2, label="Long SMA")
ax.plot('Date', 'Short sma', data=df, marker='', color='green', linewidth=2, label="Short SMA")
ax.plot('Date', 'Price', data=df, marker='', color='blue', linewidth=2, label="Price")

ax.set_xlabel("Date",fontsize=14)
ax.set_ylabel("Price",color="black",fontsize=14)
plt.legend()
plt.title('Long & short daily SMA for {0}-{1}'.format(ticker,exchange))
plt.show()

#### 4.3 Plot SMAs and daily prices with plotly

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(x=df['Date'], y=df['Long sma'],mode='lines', name='{0}-days SMA'.format(long_sma)))
fig.add_trace(go.Scatter(x=df['Date'], y=df['Short sma'],mode='lines',name='{0}-days SMA'.format(short_sma)))
fig.add_trace(go.Scatter(x=df['Date'], y=df['Price'],mode='lines',name='Price'))

fig.update_layout(title = 'Long & short SMA for {0}-{1}'.format(ticker,exchange))
fig.update_xaxes(title_text = 'Date')
fig.update_yaxes(title_text = 'Price')
fig.show()